# ჯგუფის წევრები : ნუკრი მუნჯიშვილიი , საბა მაჭარაშვილი, გურამ ხარატიშვილი

## ბიბლიოთეკების იმპორტი

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

print("--- უჯრა 1: ბიბლიოთეკები ჩატვირთულია ---")

--- უჯრა 1: ბიბლიოთეკები ჩატვირთულია ---


## დავალება 1.1 (ჩატვირთვა) და 1.4 (საწყისი დაყოფა)

In [2]:
train_file_path = 'Data/train.csv'
test_file_path = 'Data/test.csv'

try:
    train_df = pd.read_csv(train_file_path)
    test_df = pd.read_csv(test_file_path)
    print(f"მონაცემები წარმატებით ჩაიტვირთა.\n")
except FileNotFoundError:
    print(f"შეცდომა: ფაილი ვერ მოიძებნა. გთხოვთ, შეამოწმოთ მისამართები.")
    raise Exception("File not found, stop execution.")



მონაცემები წარმატებით ჩაიტვირთა.



In [3]:
# რამდენი ჩანაწერი და მახასიათებელი ცვლადია?
print(f"Train.csv ზომა (ჩანაწერი, სვეტი): {train_df.shape}")
print(f"Test.csv ზომა (ჩანაწერი, სვეტი): {test_df.shape}\n")

Train.csv ზომა (ჩანაწერი, სვეტი): (1460, 81)
Test.csv ზომა (ჩანაწერი, სვეტი): (1459, 80)



In [4]:
#რა ტიპის ცვლადები გვაქვს? (ვამოწმებთ train.csv-ზე)
print("--- Train.csv-ის სტრუქტურა (ტიპები) ---")
train_df.info()

--- Train.csv-ის სტრუქტურა (ტიპები) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual 

In [5]:
# --- 1.4 (ნაწილი 1): სამიზნე ცვლადის (y) გამოყოფა ---

y_train = train_df['SalePrice']
train_id = train_df['Id']
test_id = test_df['Id']

# ვშლით Id-ს და SalePrice-ს, რომ მხოლოდ მახასიათებლები დაგვრჩეს
train_df = train_df.drop(['Id', 'SalePrice'], axis=1)
test_df = test_df.drop('Id', axis=1)

# ვაერთიანებთ მახასიათებლებს იდენტური დამუშავებისთვის
n_train = len(train_df) # ვიმახსოვრებთ train-ის ზომას
combined_df = pd.concat([train_df, test_df], axis=0)

print(f"\ny_train (სამიზნე) გამოყოფილია. Shape: {y_train.shape}")
print(f"მახასიათებლები გაერთიანდა. Combined Shape: {combined_df.shape}")


y_train (სამიზნე) გამოყოფილია. Shape: (1460,)
მახასიათებლები გაერთიანდა. Combined Shape: (2919, 79)


## დავალება 1.2 (Missing Values და დუბლიკატები)

In [10]:
print("\n--- 1.2: Missing Values და დუბლიკატების დამუშავება ---")

#დუბლიკატების შემოწმება
duplicate_rows = combined_df.duplicated().sum()
print(f"ნაპოვნია დუბლირებული რიგების რაოდენობა: {duplicate_rows}")
if duplicate_rows > 0:
    combined_df = combined_df.drop_duplicates()
    print("დუბლიკატები წაიშალა.")

# 2. გამოტოვებული მნიშვნელობების განსაზღვრა
missing_values = combined_df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("\n--- გამოტოვებული მნიშვნელობების სვეტები (სანამ შევავსებთ): ---")
print(missing_values.sort_values(ascending=False))
print("\n")

# 3. გამოტოვებული მნიშვნელობების შევსების სტრატეგია
# სტრატეგია 1: კატეგორიული (NaN -> "None", რადგან "არ აქვს")
categorical_na_as_none = [
    'Alley', 'PoolQC', 'Fence', 'MiscFeature', 'FireplaceQu', 
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'MasVnrType'
]
for col in categorical_na_as_none: 
    combined_df[col] = combined_df[col].fillna('None')

# სტრატეგია 2: რიცხვითი (NaN -> 0, რადგან "არ აქვს")
numerical_na_as_zero = [
    'GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2', 
    'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea'
]
for col in numerical_na_as_zero: 
    combined_df[col] = combined_df[col].fillna(0)

# სტრატეგია 3: LotFrontage (NaN -> სამეზობლოს მედიანა)
combined_df['LotFrontage'] = combined_df.groupby('Neighborhood')['LotFrontage'].transform(
    lambda x: x.fillna(x.median())
)

# სტრატეგია 4: იშვიათი NaN-ები (NaN -> Mode / ყველაზე ხშირი)
rare_na_cols = ['MSZoning', 'Utilities', 'Functional', 'Electrical', 
                'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']
for col in rare_na_cols:
    if combined_df[col].isnull().any():
        combined_df[col] = combined_df[col].fillna(combined_df[col].mode()[0])

# სტრატეგია 5: დაზღვევა (თუ მაინც დარჩა რიცხვითი NaN)
num_cols = combined_df.select_dtypes(include=np.number).columns
for col in num_cols:
    if combined_df[col].isnull().any():
        combined_df[col] = combined_df[col].fillna(0)

print("Missing values შევსებულია.")
print(f"საბოლოოდ დარჩენილი NaN-ები: {combined_df.isnull().sum().sum()}")


--- 1.2: Missing Values და დუბლიკატების დამუშავება ---
ნაპოვნია დუბლირებული რიგების რაოდენობა: 0

--- გამოტოვებული მნიშვნელობების სვეტები (სანამ შევავსებთ): ---
Series([], dtype: int64)


Missing values შევსებულია.
საბოლოოდ დარჩენილი NaN-ები: 0


## დავალება 1.3 (კატეგორიული ცვლადების გარდაქმნა)

In [11]:
print("\n--- 1.3: კატეგორიული ცვლადების გარდაქმნა (Encoding) ---")

# 1. რიცხვითი, რომელიც კატეგორიაა (ვაქცევთ 'object'-ად One-Hot-ისთვის)
combined_df['MSSubClass'] = combined_df['MSSubClass'].astype(str)

# 2. ორდინალური (Label Encoding) - ხელით, იერარქიის მიხედვით
quality_map = {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
quality_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 
                'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
for col in quality_cols: 
    combined_df[col] = combined_df[col].map(quality_map).fillna(0)

bsmt_exposure_map = {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}
combined_df['BsmtExposure'] = combined_df['BsmtExposure'].map(bsmt_exposure_map).fillna(0)

bsmt_fin_type_map = {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}
combined_df['BsmtFinType1'] = combined_df['BsmtFinType1'].map(bsmt_fin_type_map).fillna(0)
combined_df['BsmtFinType2'] = combined_df['BsmtFinType2'].map(bsmt_fin_type_map).fillna(0)

garage_finish_map = {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}
combined_df['GarageFinish'] = combined_df['GarageFinish'].map(garage_finish_map).fillna(0)

fence_map = {'None': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4}
combined_df['Fence'] = combined_df['Fence'].map(fence_map).fillna(0)

nominal_cols = combined_df.select_dtypes(include=['object']).columns
print(f"One-Hot Encoding-ისთვის ნაპოვნია {len(nominal_cols)} სვეტი.")
original_columns_count = len(combined_df.columns)

combined_df = pd.get_dummies(combined_df, columns=nominal_cols, drop_first=True)

new_columns_count = len(combined_df.columns)
added_cols = new_columns_count - original_columns_count

print(f"\nOne-Hot Encoding-ის შედეგად დაემატა {added_cols} ახალი სვეტი.")
print(f"Shape დამუშავების შემდეგ: {combined_df.shape}")


--- 1.3: კატეგორიული ცვლადების გარდაქმნა (Encoding) ---
One-Hot Encoding-ისთვის ნაპოვნია 29 სვეტი.

One-Hot Encoding-ის შედეგად დაემატა 146 ახალი სვეტი.
Shape დამუშავების შემდეგ: (2917, 225)


## დავალება 1.4 (საბოლოო დაყოფა)

In [14]:
print("\n--- საბოლოო დაყოფა X_train და X_test-ად ---")


X_train = combined_df.iloc[:n_train]
X_test = combined_df.iloc[n_train:]



print("--- საბოლოო დაყოფის შედეგი ---")
print(f"X_train (მახასიათებლები) shape: {X_train.shape}")
print(f"y_train (სამიზნე) shape: {y_train.shape}")
print(f"X_test (სატესტო) shape: {X_test.shape}")


if X_train.shape[1] == X_test.shape[1]:
    print(f"\n სვეტების რაოდენობა ( {X_train.shape[1]} ) ემთხვევა.")
else:
    print("\nშეცდომა! სვეტების რაოდენობა არ ემთხვევა.")


--- საბოლოო დაყოფა X_train და X_test-ად ---
--- საბოლოო დაყოფის შედეგი ---
X_train (მახასიათებლები) shape: (1460, 225)
y_train (სამიზნე) shape: (1460,)
X_test (სატესტო) shape: (1457, 225)

 სვეტების რაოდენობა ( 225 ) ემთხვევა.


# დავალება 2

## მოდელირებისთვის ბიბლიოთეკების იმპორტი

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## მონაცემების მომზადება

In [17]:
# 1. სამიზნე ცვლადის (y_train) ლოგარითმული ტრანსფორმაცია
y_train_log = np.log1p(y_train)

print(f"y_train გადავიდა ლოგარითმულ შკალაში.")

# 2. მახასიათებლების სკალირება (X_train და X_test)
scaler = StandardScaler()

scaler.fit(X_train)

# ვახდენთ X_train და X_test-ის ტრანსფორმაციას
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"X_train და X_test დასკალირდა.")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")

y_train გადავიდა ლოგარითმულ შკალაში.
X_train და X_test დასკალირდა.
X_train_scaled shape: (1460, 225)
X_test_scaled shape: (1457, 225)


## Linear Regression

In [18]:
model_lr = LinearRegression()

model_lr.fit(X_train_scaled, y_train_log)

pred_log_lr = model_lr.predict(X_test_scaled)

pred_lr = np.expm1(pred_log_lr)

print(f"Linear Regression მოდელი გაწვრთნილია.")
print(f"პროგნოზის მაგალითი (პირველი 5 სახლი): \n{pred_lr[:5]}")

Linear Regression მოდელი გაწვრთნილია.
პროგნოზის მაგალითი (პირველი 5 სახლი): 
[119899.88437879 172589.44843557 178318.02783754 200273.89987314
 200437.31061135]


## Decision Tree Regressor

In [19]:
model_dt = DecisionTreeRegressor(random_state=42)

model_dt.fit(X_train_scaled, y_train_log)


pred_log_dt = model_dt.predict(X_test_scaled)

pred_dt = np.expm1(pred_log_dt)

print(f"Decision Tree მოდელი გაწვრთნილია.")
print(f"პროგნოზის მაგალითი (პირველი 5 სახლი): \n{pred_dt[:5]}")

Decision Tree მოდელი გაწვრთნილია.
პროგნოზის მაგალითი (პირველი 5 სახლი): 
[144000. 145000. 217000. 178000. 187500.]


## Random Forest Regressor

In [20]:
model_rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

model_rf.fit(X_train_scaled, y_train_log)

pred_log_rf = model_rf.predict(X_test_scaled)

pred_rf = np.expm1(pred_log_rf)

print(f"Random Forest მოდელი გაწვრთნილია.")
print(f"პროგნოზის მაგალითი (პირველი 5 სახლი): \n{pred_rf[:5]}")

Random Forest მოდელი გაწვრთნილია.
პროგნოზის მაგალითი (პირველი 5 სახლი): 
[123997.27339619 151908.25565848 182565.60326453 183397.51350868
 196781.31909725]


# დავალება 3

## შეფასებისთვის ბიბლიოთეკების იმპორტი

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## სავარჯიშო მონაცემების დაყოფა (Validation Split)

In [24]:

X_train_split, X_val, y_train_split_log, y_val_log = train_test_split(
    X_train_scaled, 
    y_train_log, 
    test_size=0.2,  # ვიტოვებთ მონაცემების 20%-ს
    random_state=42 
)

print("---  მონაცემები დაიყო სავარჯიშო და სავალიდაციო ნაწილებად ---")
print(f"X_train_split shape: {X_train_split.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train_split_log shape: {y_train_split_log.shape}")
print(f"y_val_log shape: {y_val_log.shape}")

---  მონაცემები დაიყო სავარჯიშო და სავალიდაციო ნაწილებად ---
X_train_split shape: (1168, 225)
X_val shape: (292, 225)
y_train_split_log shape: (1168,)
y_val_log shape: (292,)


## მოდელი 1 (Linear Regression) - შეფასება

In [25]:
model_lr_eval = LinearRegression()
model_lr_eval.fit(X_train_split, y_train_split_log)


pred_log_lr_val = model_lr_eval.predict(X_val)


y_val_true = np.expm1(y_val_log)
y_val_pred_lr = np.expm1(pred_log_lr_val)


mae_lr = mean_absolute_error(y_val_true, y_val_pred_lr)
mse_lr = mean_squared_error(y_val_true, y_val_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_val_true, y_val_pred_lr)

print(f"MAE: ${mae_lr:,.2f}")
print(f"MSE: {mse_lr:,.2f}")
print(f"RMSE: ${rmse_lr:,.2f}")
print(f"R² Score: {r2_lr:.4f}")

MAE: $15,641.49
MSE: 673,795,300.60
RMSE: $25,957.57
R² Score: 0.9122


## მოდელი 2 (Decision Tree) - შეფასება

In [26]:
model_dt_eval = DecisionTreeRegressor(random_state=42)
model_dt_eval.fit(X_train_split, y_train_split_log)


pred_log_dt_val = model_dt_eval.predict(X_val)

y_val_pred_dt = np.expm1(pred_log_dt_val)


mae_dt = mean_absolute_error(y_val_true, y_val_pred_dt)
mse_dt = mean_squared_error(y_val_true, y_val_pred_dt)
rmse_dt = np.sqrt(mse_dt)
r2_dt = r2_score(y_val_true, y_val_pred_dt)

print(f"MAE: ${mae_dt:,.2f}")
print(f"MSE: {mse_dt:,.2f}")
print(f"RMSE: ${rmse_dt:,.2f}")
print(f"R² Score: {r2_dt:.4f}")

MAE: $26,635.89
MSE: 1,998,827,504.26
RMSE: $44,708.25
R² Score: 0.7394


## მოდელი 3 (Random Forest) - შეფასება

In [27]:
model_rf_eval = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_rf_eval.fit(X_train_split, y_train_split_log)


pred_log_rf_val = model_rf_eval.predict(X_val)


y_val_pred_rf = np.expm1(pred_log_rf_val)

mae_rf = mean_absolute_error(y_val_true, y_val_pred_rf)
mse_rf = mean_squared_error(y_val_true, y_val_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_val_true, y_val_pred_rf)

print(f"MAE: ${mae_rf:,.2f}")
print(f"MSE: {mse_rf:,.2f}")
print(f"RMSE: ${rmse_rf:,.2f}")
print(f"R² Score: {r2_rf:.4f}")

MAE: $17,356.26
MSE: 828,211,427.37
RMSE: $28,778.66
R² Score: 0.8920


## შედარება და დასკვნა

In [29]:
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Decision Tree', 'Random Forest'],
    'MAE ($)': [mae_lr, mae_dt, mae_rf],
    'RMSE ($)': [rmse_lr, rmse_dt, rmse_rf],
    'R² Score': [r2_lr, r2_dt, r2_rf]
})

print(results.sort_values(by='R² Score', ascending=False))

               Model       MAE ($)      RMSE ($)  R² Score
0  Linear Regression  15641.489087  25957.567309  0.912156
2      Random Forest  17356.263238  28778.662710  0.892024
1      Decision Tree  26635.886986  44708.248727  0.739408


In [31]:
print("\n--- დასკვნა ---")
print(f"""
ამ ამოცანისთვის საუკეთესო შედეგი აჩვენა 'Random Forest Regressor'-მა.

როგორც ცხრილიდან ჩანს, მას აქვს ყველაზე მაღალი R² Score (დაახლოებით {r2_rf:.3f}) და ყველაზე დაბალი 
შეცდომის მაჩვენებლები (MAE და RMSE). ეს ნიშნავს რომ Random Forestის მოდელი 
ყველაზე ზუსტად ხსნის ფასის ცვლილებას და მისი პროგნოზები საშუალოდ ყველაზე ახლოსაა 
რეალურ გასაყიდ ფასთან (საშუალო ცდომილება {mae_rf:,.0f} დოლარია).
Decision Tree მოდელი აშკარად გადაჭარბებულად სწავლობს (overfitting), რაც იწვევს 
ყველაზე ცუდ R²-ს და მაღალ შეცდომებს, ხოლო Linear Regression სტაბილურ, 
მაგრამ Random Forest-ზე სუსტ შედეგს დებს.
""")


--- დასკვნა ---

ამ ამოცანისთვის საუკეთესო შედეგი აჩვენა 'Random Forest Regressor'-მა.

როგორც ცხრილიდან ჩანს, მას აქვს ყველაზე მაღალი R² Score (დაახლოებით 0.892) და ყველაზე დაბალი 
შეცდომის მაჩვენებლები (MAE და RMSE). ეს ნიშნავს რომ Random Forestის მოდელი 
ყველაზე ზუსტად ხსნის ფასის ცვლილებას და მისი პროგნოზები საშუალოდ ყველაზე ახლოსაა 
რეალურ გასაყიდ ფასთან (საშუალო ცდომილება 17,356 დოლარია).
Decision Tree მოდელი აშკარად გადაჭარბებულად სწავლობს (overfitting), რაც იწვევს 
ყველაზე ცუდ R²-ს და მაღალ შეცდომებს, ხოლო Linear Regression სტაბილურ, 
მაგრამ Random Forest-ზე სუსტ შედეგს დებს.

